<a href="https://colab.research.google.com/github/vlx300/kb_colab/blob/master/HTTP_Requests_for_Humans.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**Python HTTP Request Library**
**REST**: Essentially a set of conventions for structuring a WEB API (an API you interect with via HTTP: making requests to specific URLS and gathering relevant data in the Response. You leverage the several HTTP methods GET, POST, UPDATE, DELETE etc. 


![alt text](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQK4tRxuYv7nnqsPCZWnzlXJgmNrUbldBIeNiBUsGA_2UWu72Sd&s)

In [1]:
import requests 

# Make an API call  and store the response #
url = 'https://api.github.com/search/repositories?q=language:python&sort=stars'
r = requests.get(url)
print("Status code:", r.status_code)

Status code: 200


##**Web API Call alternative**

In [2]:
requests.get('https://api.github.com')

<Response [200]>

#**The Response**

A response is a powerful object for inspecting the results of a request. Make the same requet again and store the retun value in a variable so you can take a closer look at the data. Elegant

In [0]:
response = requests.get('https://api.github.com')

In this example, you've captured the "return value" of GET(), which is an instance of response, and store it in a variable called response. Now you can use response to see alot of information about the results of your GET request. 

##**Status Codes**

the first bit of information you can gather forma response is the status code. this informs you of the status of the request. For Exp. 200 means the request was successful . whereas  404 = NOT FOUND, 403 = NOT Authorized etc.. 302 = Redirect 

**Note**: *By accessing .status_code, you can see the status of the request returned from the Server.  Sometimes you might want to make decision in your code based upo the response "status code"* 

In [4]:
response.status_code # checks the status from the Server 

200

Making decisions (Other Logic) based on status code:

In [5]:
if response.status_code == 200: 
  print("Success")
elif response.status_code == 400:
  print("Not Found")
elif response.status_code == 302:
  print("Follow Redirect")
elif response.status_code == 302:
  print("Not Authorized")

Success


If your response is used in a conditional expresssion, it will evaluate to **True** if that satus code is between 200 and 400 and **False** Otherwise. so you can simply the above ike this (See Below)..  

Note: *this is possible due to __Bool__() overload method on response.  Translation, the default behaviour of response has be modified to take status code into account when determining the "Truth value" of the object* so make sure to utilize ths convenient shorthand only if you want to know if a request was "generally sucessfull" 

In [6]:
if response:
  print("Success")
else:
  print('An error has occured')

Success


Say if you dont want to check the status code like above but want to raise an exception if the request was unsuccessful only. You can using '.raise_for_status'

In [7]:
import requests
from requests import HTTPError

for url in ['http://api.github.com', 'https://api.github.com/invalid']:
  try:
    reponse = requests.get(url)

    # if the response was successful, no exception will be raised
    response.raise_for_status()
  except HTTPError as http_err:
    print(f'HTTP error has occured: {http_err}')  #python 3.6
  except Exception as err:
    print(f'Other error occured: {err}') #Python 3.6 
  else:
    print("success")


success
success


#**API Content**

The reponse of the HTTP API GET request has some valuable information i.e..Payload in the message body. Using the attributes of Response, you can view the payload in many different formats.

Bytes (.Content)

In [8]:
response = requests.get('https://api.github.com')
response.content  # give you the content in raw bytes of the response payload

b'{"current_user_url":"https://api.github.com/user","current_user_authorizations_html_url":"https://github.com/settings/connections/applications{/client_id}","authorizations_url":"https://api.github.com/authorizations","code_search_url":"https://api.github.com/search/code?q={query}{&page,per_page,sort,order}","commit_search_url":"https://api.github.com/search/commits?q={query}{&page,per_page,sort,order}","emails_url":"https://api.github.com/user/emails","emojis_url":"https://api.github.com/emojis","events_url":"https://api.github.com/events","feeds_url":"https://api.github.com/feeds","followers_url":"https://api.github.com/user/followers","following_url":"https://api.github.com/user/following{/target}","gists_url":"https://api.github.com/gists{/gist_id}","hub_url":"https://api.github.com/hub","issue_search_url":"https://api.github.com/search/issues?q={query}{&page,per_page,sort,order}","issues_url":"https://api.github.com/issues","keys_url":"https://api.github.com/user/keys","label_sea

In [9]:
response.text # convert into a string using encoding like UTF-8

'{"current_user_url":"https://api.github.com/user","current_user_authorizations_html_url":"https://github.com/settings/connections/applications{/client_id}","authorizations_url":"https://api.github.com/authorizations","code_search_url":"https://api.github.com/search/code?q={query}{&page,per_page,sort,order}","commit_search_url":"https://api.github.com/search/commits?q={query}{&page,per_page,sort,order}","emails_url":"https://api.github.com/user/emails","emojis_url":"https://api.github.com/emojis","events_url":"https://api.github.com/events","feeds_url":"https://api.github.com/feeds","followers_url":"https://api.github.com/user/followers","following_url":"https://api.github.com/user/following{/target}","gists_url":"https://api.github.com/gists{/gist_id}","hub_url":"https://api.github.com/hub","issue_search_url":"https://api.github.com/search/issues?q={query}{&page,per_page,sort,order}","issues_url":"https://api.github.com/issues","keys_url":"https://api.github.com/user/keys","label_sear

Decoding bytes to a string requires an encoding scheme. request will try to guess encoding based upon the response headers if not specified.  Set encoding BEFORE accessing .text

In [10]:
response.encoding = 'utf-8'  #optional enciding parameter.  requests infers this
response.text

'{"current_user_url":"https://api.github.com/user","current_user_authorizations_html_url":"https://github.com/settings/connections/applications{/client_id}","authorizations_url":"https://api.github.com/authorizations","code_search_url":"https://api.github.com/search/code?q={query}{&page,per_page,sort,order}","commit_search_url":"https://api.github.com/search/commits?q={query}{&page,per_page,sort,order}","emails_url":"https://api.github.com/user/emails","emojis_url":"https://api.github.com/emojis","events_url":"https://api.github.com/events","feeds_url":"https://api.github.com/feeds","followers_url":"https://api.github.com/user/followers","following_url":"https://api.github.com/user/following{/target}","gists_url":"https://api.github.com/gists{/gist_id}","hub_url":"https://api.github.com/hub","issue_search_url":"https://api.github.com/search/issues?q={query}{&page,per_page,sort,order}","issues_url":"https://api.github.com/issues","keys_url":"https://api.github.com/user/keys","label_sear

##**Serialized JSON content** 
Hey look Ma, it's JSON.....Let's view some rea-life JSON 

![alt text](https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcQt00IECBfWRzEFyW6D6dDnCvjuEt4MlTdF3WYi0IWg-_12ayXM&s)



In [11]:
response.json()  # this will show you serialized JSON content (See below).  to get a dictionary, you could take the str you received from .text and deserialize it (json.loads()) 
# much simplier to use .json(). the "type" of the return value .json() is a dictionary so you can access values i the object via the 'key'

{'authorizations_url': 'https://api.github.com/authorizations',
 'code_search_url': 'https://api.github.com/search/code?q={query}{&page,per_page,sort,order}',
 'commit_search_url': 'https://api.github.com/search/commits?q={query}{&page,per_page,sort,order}',
 'current_user_authorizations_html_url': 'https://github.com/settings/connections/applications{/client_id}',
 'current_user_repositories_url': 'https://api.github.com/user/repos{?type,page,per_page,sort}',
 'current_user_url': 'https://api.github.com/user',
 'emails_url': 'https://api.github.com/user/emails',
 'emojis_url': 'https://api.github.com/emojis',
 'events_url': 'https://api.github.com/events',
 'feeds_url': 'https://api.github.com/feeds',
 'followers_url': 'https://api.github.com/user/followers',
 'following_url': 'https://api.github.com/user/following{/target}',
 'gists_url': 'https://api.github.com/gists{/gist_id}',
 'hub_url': 'https://api.github.com/hub',
 'issue_search_url': 'https://api.github.com/search/issues?q={q

**NOTE**: you can do a alot with status codes and message bodies, but in order to get more infomration, like METADATA about the response itself, you will need to review the response headers. 

##**API Response Headers**

The repsonse headers can provide lots of useful information such as content-type of the response payload, time-limit on how long to Cache the response etc..The View these headers, access (.headers)

In [12]:
response.headers

{'Server': 'GitHub.com', 'Date': 'Thu, 26 Dec 2019 15:11:27 GMT', 'Content-Type': 'application/json; charset=utf-8', 'Transfer-Encoding': 'chunked', 'Status': '200 OK', 'X-RateLimit-Limit': '60', 'X-RateLimit-Remaining': '55', 'X-RateLimit-Reset': '1577376686', 'Cache-Control': 'public, max-age=60, s-maxage=60', 'Vary': 'Accept', 'ETag': 'W/"307bf49b7e1b8e6fe4ef622d609665e1"', 'X-GitHub-Media-Type': 'github.v3; format=json', 'Access-Control-Expose-Headers': 'ETag, Link, Location, Retry-After, X-GitHub-OTP, X-RateLimit-Limit, X-RateLimit-Remaining, X-RateLimit-Reset, X-OAuth-Scopes, X-Accepted-OAuth-Scopes, X-Poll-Interval, X-GitHub-Media-Type', 'Access-Control-Allow-Origin': '*', 'Strict-Transport-Security': 'max-age=31536000; includeSubdomains; preload', 'X-Frame-Options': 'deny', 'X-Content-Type-Options': 'nosniff', 'X-XSS-Protection': '1; mode=block', 'Referrer-Policy': 'origin-when-cross-origin, strict-origin-when-cross-origin', 'Content-Security-Policy': "default-src 'none'", 'Con

In [13]:
response.headers['content-Type']  # Accesing the key to content-type METADATA header.   NOT Case sensitive

'application/json; charset=utf-8'

In [14]:
response.headers['X-RateLimit-Remaining']

'55'

##**Maniupulating Query parameters (params)**

One common way to customize a GET request is to pass values thru Query String parameters in the URL. Todo this, you pass data to *params* 

query strings are useful for "parameterizing" GET requests. 

In [15]:
import requests

# search github's reposititories for requests 
response = requests.get('https://api.github.com/search/repositories', params=[('q', 'requests+language:python')], )
# return sever status
reponse.status_code

404

In [16]:
# you can pass values as Bytes
requests.get('https://api.github.com/search/repositories', params=b'q=requests+language:python')

<Response [200]>

##**Request Headers**

to customize headers,  you pass a **dictionary of HTTP headers** to GET() using the *'headers'* parameter. for exp, ypou can change your previous search request to highlight matching search terms in the results by specifying the "text-match media type in the *accept* header.

In [17]:
import requests

response = requests.get('https://api.github.com/search/repositories',
                        params={'q': 'requests+language:python'}, 
                        headers={'Accept': 'application/vnd.github.com.v3.text-match+json'},)

# View the new text matches array which provides informtion 
# about your search terms within the results
json_response = response.json()
repository = json_response['items'][0]
print(f'text matches: {repository["text_matches"]}') 

text matches: [{'object_url': 'https://api.github.com/repositories/4290214', 'object_type': 'Repository', 'property': 'description', 'fragment': 'Requests + Gevent = <3', 'matches': [{'text': 'Requests', 'indices': [0, 8]}]}]


the accept header tells the server what content you application can handle. in thid case, since you're expecting the matching search terms to be highllighted , your using the header value "*application/vnd/github.v3.text-match+json*" which is a proprietart github accept header where the content is a special JSON format. 

before you learn ways to customize requests, Lets broden the horizon by exploring other HTTP methods. 

##**Other HTTP Methods**

Aside from GET, other popular HTTP methods include POST, PUT, DELETE, HEAD, PATCH and OPTIONS headers.  Requests provides a methodwith a similar signature to GET(), for each of these methods. 

Each function call makes a request to the HTTPbin service using the corresponding HTTP method. for each method, you can inspect each method as before. (*See Below*)

In [18]:
requests.post('https://httpbin.org/post', data={'key':'value'})
requests.put('https://httpbin.org/put', data={'key':'value'})
requests.delete('https://httpbin.org/delete') 
requests.head('https://httpbin.org/get')
requests.patch('https://httpbin.org/patch', data={'key':'value'})
requests.options('https://httpbin.org/get')

<Response [200]>

In [19]:
response = requests.head('https://httpbin.org/get')
response.headers['content-type']

'application/json'

In [20]:
response = requests.delete('https://httpbin.org/delete')
json_response = response.json()
json_response['args']

{}

#**Message Body**

Headers, response bodies, status codes and more are returned in the response for each method. Now lets tke a closer look at POST, PUT and PATCH HTTP methods. According to the HTTP spec, POST/PUT and PATCH pass their data thru the body of an HTTP message instead of the URL. Using requests, you pass the payload to the corresponding ***data*** parameter. 


Data takes a dictionary, list of tuples or  file-like object. you will want to adapt the data you send in thr message body to the specific needs of the service your interacting with.  

Exp: if your requests "*content-Type*" is "*application/x-www-form-urlencoded*", you can send data as a dictionary 

In [21]:
requests.post('https://httpbin.org/post', data={'key':'value'})

<Response [200]>

In [22]:
requests.post('https://httpbin.org/post', data=[('key', 'value')])

<Response [200]>

If however, you need to send ***JSON data*** , you can use the json parameter. Whe youn pass JSON data via json , requests will serialize tour data and add correct "Content-Type" header for you. 

In [23]:
response = requests.post('https://httpbin.org/post', json={'key':'value'})
json_response = response.json()
json_response['data']

'{"key": "value"}'

In [24]:
json_response['headers']['Content-Type']

'application/json'

You can see from the response that the server received your request data and headers as you sent,  requests also provides this infomration to you in a form of a PreparedRequests. 

#**Inspecting the Request**

When you make a request, the requests library prepares the request before actually sending to the destination server, preperation includes things such as "**header-validation**" and **serialization** of the data (*JSON Content)*. 

You can access the **PreparedRequests**  by accessing .request

In [25]:
response = requests.post('https://httpbin.org/post', json={'key':'value'})
response.request.headers['Content-type']

'application/json'

In [26]:
response.request.url

'https://httpbin.org/post'

In [27]:
response.request.body

b'{"key": "value"}'

Inspecting the rquest gives tyou access to all kinds of information about the request being made:

*   Payload
*   URL
*   Headers
*   Authenication and more






#**API Authentication**